In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import json
import requests
import re
import time
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
import pandas as pd
import requests
import io
import re
import telebot
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
class HH_vacancies:
    def __init__(self, ):
        res_df  = self.search_areas_ids()
        self.areas_ids = res_df['id_y'].unique().tolist()
        
    def search_areas_ids(self):
        #строка запроса, где 113 - код России
        url = 'https://api.hh.ru/areas/113'
        # отправляю запрос и полученный ответ преобразовываю в json
        result = requests.get(url).json()['areas']
        # преобразовываю в DataFrame для удобства дальнейшей работы с данными
        df = pd.json_normalize(result)
        # произвожу поиск вложенных areas для parent_ids
        areas_list = []
        for item in df['areas'].tolist():
            areas_list += item
        df2 = pd.json_normalize(areas_list)
        res_df = df.drop('areas', axis=1).merge(
                                                df2.drop('areas', axis=1),
                                                left_on='id',
                                                right_on='parent_id',
                                                how='inner'
                                            )
        # удаляю колонки, дублирующие данные
        res_df = res_df.drop(columns = ['parent_id_x','parent_id_y'], axis=1)
        return res_df

    def search_vacancies_ids(self, start, stop, filename):
        del_columns = [
            'address', 'response_url', 'sort_point_distance', 'relations',
            'contacts', 'working_days', 'working_time_intervals', 'working_time_modes',
            'accept_temporary', 'address.building', 'address.description',
            'address.metro', 'address.metro_stations', 'employer.logo_urls.240',
            'employer.logo_urls.90', 'employer.logo_urls.original', 'address.metro.station_name',
            'address.metro.line_name', 'address.metro.station_id', 'address.metro.line_id',
            'address.metro.lat', 'address.metro.lng', 'salary', 'insider_interview.id',
            'insider_interview.url'
        ]
        res_frame = pd.DataFrame()
        for area_id in tqdm(self.areas_ids[start:stop]):
            try:
                for page in range(1,100):
                    url = f'https://api.hh.ru/vacancies?page={page}&per_page=100&area={area_id}'
                    time.sleep(0.5)
                    req = requests.get(url).json()
                    new_frame = pd.json_normalize(req['items'])
                    res_frame = pd.concat([res_frame, new_frame])
                    if len(req['items'])==0:
                        break
            except Exception as error:
                continue
        res_frame = res_frame.drop(columns = del_columns)
        res_frame.to_csv(filename, sep=';', index = False)
    
    def get_vacancies_data(self, vacancies_ids, left, right):
        res_df = pd.DataFrame()
        count = 0
        columns = [
            'id','name','description','key_skills','specializations','professional_roles',
            'published_at','apply_alternate_url','alternate_url','area.id','area.name',
            'salary.from','salary.to','salary.currency','experience.id','experience.name',
            'schedule.name','employment.name','employer.id','employer.name','address.city',
            'address.street','address.building','address.lat','address.lng','address.raw']
        for vacancy_id in tqdm(vacancies_ids[left : right]):
            try:
                url = f'https://api.hh.ru/vacancies/{vacancy_id}'
                res = requests.get(url)
                if res.status_code == 200 and len(res.text)>10:
                    df = pd.json_normalize(res.json())
                    res_df = pd.concat([res_df, df[columns]])
                    count += 1
                    if count == 100:
                        time.sleep(5)
                        count = 0
                else:
                    time.sleep(2)
                    raise Exception('Not 200 status')
            except Exception as error:
                continue
        res_df.to_excel(f'{left}-{right}.xlsx')

In [ ]:
class Text_worker:
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        additional_stopwords = ['quot']
        self.del_words = stopwords.words('russian') + additional_stopwords
    
    def del_tags(self, text):
        """
        Метод для очистки входного текста от html-тегов.
        input:  text - текст для очистки.
        output: очищенная строка.
        """
        return re.sub('\<.{1,9}\>',' ',text)
    
    def clean_text(self, dirty_text):
        """
        Метод для очистки текста.
        input:  dirty_text - текст для очистки.
        output: result_text - токенизированный текст без стоп-слов и в нормальной форме.
        """
        sub_text = re.sub('[^А-Яа-яA-Za-z\s\|]|\s{2,}',' ', self.del_tags(dirty_text) )
        n_sub_text = re.sub('\s{2,10}',' ', sub_text).strip().lower()
        tokens = word_tokenize(str(n_sub_text))
        clean_tokens = tuple(map(lambda x: self.morph.parse(x)[0].normal_form if x not in self.del_words else '', tokens ))
        result_text = ' '.join(clean_tokens)
        return result_text

In [ ]:
bot = telebot.TeleBot('5689764563:AAHuZaNiVga_NBwlst2EELS46Qevxe_QOoY')

text_worker = Text_worker()
url = 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1rNdkRAa8ilISD2mrrveDJSeV5iz4f8hW'
df = pd.read_csv(url, sep='|')
df['description_2'] = df['description_2'].apply(lambda x: re.sub(r'\s{2,10}',' ',str(x)) )
docs = df['description_2'].astype(str).tolist()
card_docs = [TaggedDocument(doc.split(' '), [i]) for i, doc in enumerate(docs)]
model = Doc2Vec(vector_size=64, window=2, min_count=10, workers=8, epochs = 40)
model.build_vocab(card_docs)
model.train(card_docs, total_examples=model.corpus_count, epochs=model.epochs)
print('Модель натренирована')
# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    greeting = """
    Здравствуйте.
    Для продолжения работы отправьте сообщение с текстовым описанием вакансии для поиска.
    """
    bot.send_message(m.chat.id, greeting)

# Получение сообщений от юзера
@bot.message_handler(content_types=["text"])
def handle_text(message):
    try:
        splited_text = re.sub('\s{2,5}',' ', text_worker.clean_text( str(message)) ).split(' ')
        vector_to_search = model.infer_vector(splited_text)
        sim_documents = model.docvecs.most_similar([vector_to_search], topn=5)
        
        urls = [str(df['alternate_url'].iloc[s[0]]) for s in sim_documents]
        urls_string = '\n'.join(urls)
        res_message = f'Ссылки на наиболее похожие вакансии:\n{urls_string}'
        bot.send_message(message.chat.id, res_message)
    except Exception as error:
        print(error)
        bot.send_message(message.chat.id, 'При попытке запроса произошла ошибка, повторите попытку...')

bot.polling(none_stop=True, interval=0)

In [ ]:
test_rezume = """АО НИПИГАЗ
Эксперт СМР и КИП
Организация строительства АГПЗ
Организация строительного контроля в структуре заказчика;
Составление графиков выполнения работ и контроль их реализации;
Оперативное планирование выполнения СМР подрядными организациями;
Проведение входного контроля проектной и рабочей документации;
Проверка факта выполненных объемов работ;
Проверка и подписание исполнительной документации;
Визирование актов выполненных работ;
Участие в проведении входного контроля МТР;
Контроль исполнения договорных обязательств подрядными организациями;
Ведения деловой переписки с подрядными организациями в части исполнения обязательств по строительству и контролю выполненных работ
Взаимодействия с подрядными организациями в части организации строительного производства;
Координации подрядных организаций в вопросах связанных с исполнением договорных обязательств;
Участие в совещаниях с подрядными организациями по вопросам, связанным со строительным производством;
Имею высшее техническое образование "Управление и информатика в технологических системах".
"""